In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Prepare Data

In [ ]:
# fn = '/Users/anya/Documents/ML4HC/mimic-iv-3.1/hosp/diagnoses_icd.csv.gz'
# count=1

# ad_diagnosis = pd.DataFrame({'subject_id':[], 'hadm_id': [], 'icd_code':[]})
# adrd_codes = ['F0150', 'F0280', 'F0281', 'F0390', 'F0391', 'G300', 'G301', 'G308', 'G309', 'G3101',
#              'G3109', 'G3181', 'G3182', 'G3183', 'G3184', 'G3185', 'G3189', 'G319', 'R4181']

# f1 = gzip.open(fn, 'rt')
# for line  in f1:
#     ln = line.strip().split(',')
#     subject_id = ln[0]
#     if subject_id != 'subject_id':
#         hadm_id = ln[1]
#         icd_code = ln[3]
#         icd_version = int(ln[4])
#         if icd_version == 10:
#             if icd_code in adrd_codes:
#                 new = pd.DataFrame({'subject_id': [subject_id], 'hadm_id': [hadm_id], 'icd_code': [icd_code]})
#                 ad_diagnosis = pd.concat([ad_diagnosis, new])
# f1.close()
# ad_diagnosis.to_csv('data/1_ad_diagnosis.csv', header=True, index=False)

In [ ]:
# fn = '/Users/anya/Documents/ML4HC/mimic-iv-3.1/hosp/patients.csv.gz'

# pts = {}
# f1 = gzip.open(fn, 'rt')
# for line  in f1:
#     ln = line.strip().split(',')
#     subject_id = ln[0]
#     if subject_id != 'subject_id':
#         gender = ln[1]
#         anchor_age = ln[2]
#         anchor_year = ln[3]
#         pts[subject_id] = [gender, anchor_age, anchor_year]
# f1.close()
# len(pts)

# fn = '/Users/anya/Documents/ML4HC/mimic-iv-3.1/hosp/admissions.csv.gz'

# demographics = pd.DataFrame({'subject_id': [], 'hadm_id': [], 'admityear': [], 'admitmonth': [], 'admitday': [], 'admittime': [],
#                              'gender': [], 'age': [], 'admission_type': [], 'insurance': [], 'language': [], 'marital_status': [], 'race': []})

# f1 = gzip.open(fn, 'rt')
# for line  in f1:
#     ln = line.strip().split(',')
#     subject_id = ln[0]
#     if subject_id in pts:
#         hadm_id = ln[1]
#         admittime = ln[2]
#         admission_type = ln[5]
#         insurance = ln[9]
#         language = ln[10]
#         marital_status = ln[11]
#         race = ln[12]
#         new = pd.DataFrame({'subject_id': [subject_id], 'hadm_id': [hadm_id], 'admittime': [admittime], 'admission_type': [admission_type], 'insurance': [insurance],
#                             'language': [language], 'marital_status': [marital_status], 'race': [race]})
#         new[['admitdate', 'admittime']] = new['admittime'].str.split(' ', expand=True)
#         new[['admityear', 'admitmonth', 'admitday']] = new['admitdate'].str.split('-', expand=True)
#         new = new.drop(columns='admitdate')
#         gender, anchor_age, anchor_year = pts[subject_id]
#         age = int(new['admityear'][0]) - int(anchor_year) + int(anchor_age)
#         # if age >= 65:
#         new[['gender', 'age']] = [gender, age]
#         demographics = pd.concat([demographics, new])
# f1.close()
# demographics.to_csv('data/2_demographics.csv', index=False, header=True)
# demographics


## Merge Data

In [ ]:
ad_codes = ['G300', 'G301', 'G308', 'G309']
agg_funcs = {'adrd_icd': lambda x: ','.join(x.dropna().astype(str)),
             'adrd': 'max',
             'ad': 'max'}

diag = pd.read_csv('data/1_ad_diagnosis.csv')
diag.columns = ['subject_id', 'hadm_id', 'adrd_icd']
diag['adrd'] = 1
diag['ad'] = [1 if x in ad_codes else np.nan for x in diag['adrd_icd']]
diag = diag.groupby(['subject_id', 'hadm_id'], as_index=False).agg(agg_funcs)
ad_flag = diag.groupby('subject_id')['ad'].max()
diag['case_status'] = diag['subject_id'].map(ad_flag)
print('Maximum number of unique patient (ADRD): %s' % len(diag['subject_id'].unique()))
temp = diag[diag['ad'] == 1]
print('Maximum number of unique patients (AD): %s' % len(temp['subject_id'].unique()))
diag

In [ ]:
race_map1 = {'WHITE': 'White',
            'BLACK/AFRICAN AMERICAN': 'African American',
            'UNKNOWN': 'Other',
            'WHITE - OTHER EUROPEAN': 'White',
            'WHITE - RUSSIAN': 'White',
            'OTHER': 'Other',
            'ASIAN - CHINESE': 'Asian',
            'HISPANIC/LATINO - PUERTO RICAN': 'Hispanic/Latino',
            'BLACK/CARIBBEAN ISLAND': 'Carribean Island',
            'BLACK/CAPE VERDEAN': 'Cape Verdean',
            'UNABLE TO OBTAIN': 'Other',
            'HISPANIC/LATINO - DOMINICAN': 'Hispanic/Latino',
            'ASIAN': 'Asian',
            'WHITE - EASTERN EUROPEAN': 'White',
            'BLACK/AFRICAN': 'African',
            'PORTUGUESE': 'White',
            'HISPANIC/LATINO - CUBAN': 'Hispanic/Latino',
            'ASIAN - SOUTH EAST ASIAN': 'Asian',
            'AMERICAN INDIAN/ALASKA NATIVE': 'Native American',
            'HISPANIC/LATINO - COLUMBIAN': 'Hispanic/Latino',
            'HISPANIC/LATINO - HONDURAN': 'Hispanic/Latino',
            'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'Pacific Islander',
            'ASIAN - ASIAN INDIAN': 'Indian',
            'HISPANIC/LATINO - SALVADORAN': 'Hispanic/Latino',
            'PATIENT DECLINED TO ANSWER': 'Other',
            'HISPANIC/LATINO - CENTRAL AMERICAN': 'Hispanic/Latino',
            'HISPANIC/LATINO - GUATEMALAN': 'Hispanic/Latino',
            'SOUTH AMERICAN': 'South American',
            'ASIAN - KOREAN': 'Asian',
            'HISPANIC/LATINO - MEXICAN': 'Hispanic/Latino',
            'WHITE - BRAZILIAN': 'White',
            'HISPANIC OR LATINO': 'Hispanic/Latino',
            'MULTIPLE RACE/ETHNICITY': 'Other'}

# group "black" with African American, Indian with Asian, and all other groups with other
race_map2 = {'White': 'White',
             'African American': 'African American',
             'Other': 'Other',
             'Hispanic/Latino': 'Hispanic/Latino',
             'Asian': 'Asian',
             'Cape Verdean': 'African American',
             'Carribean Island': 'African American',
             'African': 'African American',
             'Indian': 'Asian',
             'South American': 'Other',
             'Native American': 'Other',
             'Pacific Islander': 'Other'
             }

# group African with African American, Indian with Asian, and all other groups with other
race_map3 = {'White': 'White',
             'African American': 'African American',
             'Other': 'Other',
             'Hispanic/Latino': 'Hispanic/Latino',
             'Asian': 'Asian',
             'Cape Verdean': 'Other',
             'Carribean Island': 'Other',
             'African': 'African American',
             'Indian': 'Asian',
             'South American': 'Other',
             'Native American': 'Other',
             'Pacific Islander': 'Other'
             }

# just group smaller groups with other
race_map4 = {'White': 'White',
             'African American': 'African American',
             'Other': 'Other',
             'Hispanic/Latino': 'Hispanic/Latino',
             'Asian': 'Asian',
             'Cape Verdean': 'Other',
             'Carribean Island': 'Other',
             'African': 'Other',
             'Indian': 'Other',
             'South American': 'Other',
             'Native American': 'Other',
             'Pacific Islander': 'Other'
             }

insurance_map = {'Medicare': 'Medicare',
                 'Medicaid': 'Medicaid',
                 'Private': 'Other',
                 'Other': 'Other',
                 'No charge': 'Other',
                 'Unknown': 'Other'}

demo = pd.read_csv('data/2_demographics.csv')
demo['marital_status'] = demo['marital_status'].fillna('UNKNOWN')
demo['insurance'] = demo['insurance'].fillna('Other')
demo['insurance_group'] = demo['insurance'].map(insurance_map)
demo['language'] = demo['language'].fillna('Unknown')
demo['language_group'] = ['English' if x == 'English' else 'Non-English' for x in demo['language']]
demo['race_group1'] = demo['race'].map(race_map1)
demo['race_group2'] = demo['race_group1'].map(race_map2)
demo['race_group3'] = demo['race_group1'].map(race_map3)
demo['race_group4'] = demo['race_group1'].map(race_map4)
demo

In [ ]:
print(demo.shape)
print(diag.shape)
df1 = pd.merge(demo, diag, how='outer', on=['subject_id', 'hadm_id'])
print(len(df1[df1['ad']==1]['subject_id'].unique()))
df1

In [ ]:
print(len(df1[df1['adrd']==1]['subject_id'].unique()))
print(len(df1[df1['ad']==1]['subject_id'].unique()))

### Forward fill diagnosis

In [ ]:
# fix situations where ICD recorded as AD for earlier visit but not later visits
# assume that if AD diagnosed once it is diagnosed for later visits
df1_sorted = df1.sort_values(by=['subject_id', 'admityear', 'admitmonth', 'admitday', 'admittime'])
df1_sorted['ad'] = df1_sorted.groupby('subject_id')['ad'].ffill().fillna(0).astype(int)
df1_sorted['adrd'] = df1_sorted.groupby('subject_id')['adrd'].ffill().fillna(0).astype(int)
df1_sorted['adrd_icd'] = df1['adrd_icd'].fillna('NotAD')

ad_flag = df1_sorted.groupby('subject_id')['ad'].max()
df1_sorted['case_status'] = df1_sorted['subject_id'].map(ad_flag)
adrd_flag = df1_sorted.groupby('subject_id')['adrd'].max()
df1_sorted['adrd_status'] = df1_sorted['subject_id'].map(adrd_flag)
df1_sorted.to_csv('data/3_demographics_diagnosis.csv', index=False, header=True)
df1_sorted

In [ ]:
print(len(df1_sorted[df1_sorted['adrd']==1]['subject_id'].unique()))
print(len(df1_sorted[df1_sorted['ad']==1]['subject_id'].unique()))

In [ ]:
df1_sorted.isnull().any()

In [ ]:
df1_sorted[df1_sorted['subject_id']==16221825]

In [ ]:
print(len(df1_sorted[df1_sorted['adrd_status']==1]['subject_id'].unique()))
print(len(df1_sorted[df1_sorted['case_status']==1]['subject_id'].unique()))

### Discharge notes

```
awk 'BEGIN {FS=","; OFS="\t"} /^[0-9]{4,}/ {print $1,$2,$3,$4,$5,$6,$7}' discharge.csv > discharge_meta.tsv
```


In [ ]:
# want 331,794 rows - missing 1?
notes = pd.read_csv('/Users/anya/Documents/ML4HC/mimic-iv-3.1/notes/discharge_meta.tsv', sep='\t', header=None)
notes.columns = ['note_id', 'subject_id', 'hadm_id', 'note_type', 'note_seq', 'chart_time', 'store_time']
notes = notes[notes['note_type'] == 'DS']
notes['subject_id'] = notes['subject_id'].astype(int)
notes['hadm_id'] = notes['hadm_id'].astype(int)
notes

In [ ]:
df2 = pd.merge(df1_sorted, notes, how='left', on=['subject_id', 'hadm_id'])
df2 = df2[df2['subject_id'].isin(notes['subject_id'])]
df2.to_csv('data/4_demographics_diagnosis_dsnotes.csv', index=False, header=True)
print(len(df2[df2['case_status']==1]['subject_id'].unique()))
df2

In [ ]:
df2.isnull().any()

### ADRD vs. AD

In [ ]:
print(len(df2[df2['adrd_status']==1]['subject_id'].unique()))
print(len(df2[df2['case_status']==1]['subject_id'].unique()))

## Matching

Remember to exclude patients with ADRD from control selection  
Code adapted from Sierra

In [ ]:
match_df = df2[['subject_id', 'hadm_id', 'admityear', 'admitmonth', 'admitday', 'admittime',
                'gender', 'age', 'admission_type', 'insurance_group', 'language_group', 'marital_status', 'race_group1',
                'adrd', 'ad', 'case_status', 'adrd_status']]
match_df.isnull().any()

In [ ]:
cases = match_df[match_df['case_status']==1]
cases_counts = cases.groupby('subject_id')['hadm_id'].count()
cutoff = np.percentile(cases_counts, 95)
print('The 95th percentile cutoff is %s' % cutoff)
plt.hist(cases_counts, bins=100)
plt.title('Hospital admissions per patient')
plt.show()

In [ ]:
ctrls = match_df[match_df['adrd_status'] == 0]
ctrls_counts = ctrls.groupby('subject_id')['hadm_id'].count()
cutoff = np.percentile(ctrls_counts, 95)
print('The 95th percentile cutoff is %s' % cutoff)
plt.hist(ctrls_counts, bins=100)
plt.title('Hospital admissions per patient')
plt.show()

In [ ]:
def select_row(group):
    ad1 = group[group['ad'] == 1]
    if not ad1.empty:
        return(ad1.loc[ad1['admitdate'].idxmin()])
    else:
        return(group.loc[group['admitdate'].idxmax()])

clean_df = match_df.copy()
clean_df['admitdate'] = pd.to_datetime(clean_df[['admityear', 'admitmonth', 'admitday']].rename(
    columns={'admityear': 'year', 'admitmonth': 'month', 'admitday': 'day'}))

# clean the cases so only contain first entry of AD
print('CASES')
cases = clean_df[match_df['case_status'] == 1]
print(cases.shape)
cases = cases.groupby('subject_id', group_keys=False).apply(select_row)
print(cases.shape)
cases_subjects = cases['subject_id'].unique()
print(len(cases_subjects))
cases['pheno'] = 1

# ensure there are no cases in controls
print('CONTROLS')
ctrls = clean_df[match_df['adrd_status'] == 0]
print(ctrls.shape)
# ctrls = ctrls[ctrls['age']>=65]
# print(ctrls.shape)
ctrls = ctrls[~ctrls['subject_id'].isin(cases_subjects)]
print(ctrls.shape)
ctrls['pheno'] = 0

# concat cases and controls
merged_df = pd.concat([cases, ctrls], ignore_index=True)
merged_df

In [ ]:
treatment_col = 'pheno'
covariates = ['gender', 'race_group1', 'age', 'admission_type', 'marital_status', 'language_group', 'insurance_group']

### Before matching

In [ ]:
fig, axes = plt.subplots(len(covariates), 1, figsize=(8, 6 * len(covariates)))

for i, covariate in enumerate(covariates):
    sns.histplot(data=merged_df, x=covariate, hue=treatment_col, kde=True, ax=axes[i], element="step")
    axes[i].set_xlabel(covariate, fontsize=12)
    axes[i].set_ylabel("Density", fontsize=12)
    axes[i].tick_params(axis='x', rotation=90) # Rotate x-axis labels vertically

plt.tight_layout()
plt.savefig(f'figures/{treatment_col}_before_distribution.png')
plt.show()

### Algorithm

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import BallTree

In [ ]:
df3 = merged_df[covariates]
df3[treatment_col]=merged_df[treatment_col]

# Encode categorical variables
df3 = pd.get_dummies(df3, columns= ["gender", "race_group1",'admission_type',"marital_status", 'language_group', "insurance_group"], drop_first=True)

# Calculate propensity scores
X = df3.drop(columns=[treatment_col])
y = df3[treatment_col]

ps_model = LogisticRegression(max_iter=1000, random_state=42)
ps_model.fit(X, y)
df3["propensity_score"] = ps_model.predict_proba(X)[:, 1]
df3["subject_id"]=merged_df["subject_id"]

# 1:1 matching with caliper
treated = df3[df3[treatment_col] == 1]
controls = df3[df3[treatment_col] == 0]

In [ ]:
def ultra_fast_1to1_matching_with_exclusion(treated, controls, caliper=0.2):
    """
    Optimized 1:1 matching with no control reuse and exclusion of all rows
    with the same subject_id when a control is matched.
    """
    # Convert to numpy arrays for speed
    treated_ps = treated["propensity_score"].values.reshape(-1, 1)
    control_ps = controls["propensity_score"].values.reshape(-1, 1)
    control_indices = controls.index.values
    control_subject_ids = controls["subject_id"].values  # Track subject_ids

    # Standardize
    scaler = StandardScaler()
    treated_ps_std = scaler.fit_transform(treated_ps)
    control_ps_std = scaler.transform(control_ps)

    # Build BallTree for fast radius searches
    tree = BallTree(control_ps_std)

    # Track used controls with boolean array (faster than set)
    used_controls = np.zeros(len(controls), dtype=bool)
    matches = []

    # Process in random order to avoid quality bias
    rng = np.random.default_rng(42)
    treated_order = rng.permutation(len(treated))

    for i in treated_order:
        ps = treated_ps_std[i]
        # Find all potential matches within caliper (radius search)
        match_indices = tree.query_radius([ps], r=caliper)[0]

        # Mask for unused controls
        available_mask = ~used_controls[match_indices]
        available_indices = match_indices[available_mask]

        if len(available_indices) > 0:
            # Find closest available control (using numpy argmin)
            distances = np.abs(control_ps_std[available_indices].flatten() - ps[0])
            best_idx = available_indices[np.argmin(distances)]

            # Mark ALL rows with this subject_id as used
            matched_subject_id = control_subject_ids[best_idx]
            used_controls[control_subject_ids == matched_subject_id] = True

            matches.append({
                'treated_idx': treated.index[i],
                'control_idx': control_indices[best_idx]
            })

    return matches

# Usage
matches = ultra_fast_1to1_matching_with_exclusion(treated, controls, caliper=0.2)

# Create matched dataset (vectorized operations)
treated_matched = treated.loc[[m['treated_idx'] for m in matches]]
controls_matched = controls.loc[[m['control_idx'] for m in matches]]
matched_df = pd.concat([treated_matched, controls_matched])

# Add pair IDs in one operation
matched_df['pair_id'] = np.repeat(np.arange(len(matches)), 2)
len(matched_df)

In [ ]:
# Check balance
def check_balance(df, covariates):
    balance = {}
    for var in covariates:
        if var in df.columns:
            t_mean = df[df[treatment_col] == 1][var].mean()
            c_mean = df[df[treatment_col] == 0][var].mean()
            pooled_std = np.sqrt((df[df[treatment_col] == 1][var].var() +
                                df[df[treatment_col] == 0][var].var()) / 2)
            balance[var] = abs((t_mean - c_mean) / pooled_std)
    return balance

balance_report = check_balance(matched_df, covariates)
print("Balance Report (SMD < 0.1 is good):")
for var, smd in balance_report.items():
    print(f"{var}: {smd:.3f}")

print(f"\nMatched {len(treated_matched)} treated to {len(controls_matched)} controls")

In [ ]:
# prompt: get the rows of df_selected with the same row index as in all_matched_data

# Get the indices of the matched rows
matched_indices = matched_df.index

# Select rows from df_selected using these indices
df_matched_selected = merged_df.loc[matched_indices]

# Now df_matched_selected contains the rows from df_selected that were matched
# and have the same row index as in all_matched_data.
df_matched_selected
df_matched_selected.to_csv(f'data/5_matched_{treatment_col}.csv')
print(len(df_matched_selected))
df_matched_selected

In [ ]:
# prompt: check if there's duplicative value for subject_id

# Assuming 'df' is your DataFrame as defined in the provided code.
duplicate_subjects = df_matched_selected[df_matched_selected.duplicated(subset=['subject_id'], keep=False)]

if not duplicate_subjects.empty:
    print("Duplicate subject IDs found:")
    print(duplicate_subjects[['subject_id']])
else:
    print("No duplicate subject IDs found.")

# prompt: check if df_matched_selected has duplicative index, if yes return index

# Check for duplicate indices
duplicate_indices = df_matched_selected.index[df_matched_selected.index.duplicated(keep=False)]

if not duplicate_indices.empty:
  print("Duplicated indices found:")
  print(duplicate_indices)
else:
  print("No duplicate indices found.")


In [ ]:
fig, axes = plt.subplots(len(covariates), 1, figsize=(8, 6 * len(covariates)))

for i, covariate in enumerate(covariates):
    sns.histplot(data=df_matched_selected, x=covariate, hue=treatment_col, kde=True, ax=axes[i], element="step")
    axes[i].set_xlabel(covariate, fontsize=12)
    axes[i].set_ylabel("Density", fontsize=12)
    axes[i].tick_params(axis='x', rotation=90) # Rotate x-axis labels vertically

plt.tight_layout()
plt.savefig(f'figures/{treatment_col}_after_distribution.png')
plt.show()